# Analysing the delays in a morning peak

This notebook takes a all the raw data from a single delay, and determines the number of delays, and the worst delay for a set time period in the day.

In [1]:
import zipfile
import io

In [2]:
file_name = '20190123.zip'
with open(file_name, "rb") as f:
    z = zipfile.ZipFile(io.BytesIO(f.read()))

z.extractall()
print("Extracted file " + file_name)

Extracted file 20190123.zip


We have a few directories to dive into before we get to the good stuff...

In [3]:
import glob
data_path = 'home/pi/sydney-transport-tracker/data/raw/20190123/'
delay_files_count = len(glob.glob(data_path + '*.pickle'))
print('We have ' + str(delay_files_count) + ' delay files')
timetable_files = glob.glob(data_path + '*.txt')
print('The timetable files are: ' + ', '.join(timetable_files))

We have 720 delay files
The timetable files are: home/pi/sydney-transport-tracker/data/raw/20190123/stop_times.txt, home/pi/sydney-transport-tracker/data/raw/20190123/shapes.txt, home/pi/sydney-transport-tracker/data/raw/20190123/stops.txt, home/pi/sydney-transport-tracker/data/raw/20190123/calendar.txt, home/pi/sydney-transport-tracker/data/raw/20190123/trips.txt, home/pi/sydney-transport-tracker/data/raw/20190123/agency.txt, home/pi/sydney-transport-tracker/data/raw/20190123/routes.txt


## What services are running today?
The services are determined in `calendar.txt`, with those service IDs used to filter out everything in `trips.txt`. From there we have trip IDs, which can be used to filter out the scheduled stop times in `stop_times.txt`.

`calendar.txt` -> `service_id` -> `trips.txt` -> `trip_id` -> `stop_times.txt` -> `arrival_time`, `departure_time`

In [4]:
import datetime
import pandas as pd
import csv

day_of_analysis = 'wednesday'
date_of_analysis = datetime.datetime.strptime('20190123', "%Y%m%d").date()
todays_services = []

with open(data_path + 'calendar.txt', mode='r', encoding='utf-8-sig') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if row[day_of_analysis] == '1':
            start_date = datetime.datetime.strptime(row['start_date'], "%Y%m%d").date()
            end_date = datetime.datetime.strptime(row['end_date'], "%Y%m%d").date()
            if start_date <= date_of_analysis <= end_date:
                todays_services.append(row['service_id'])

print("Todays services = " + ', '.join(todays_services))

Todays services = 1260.122.112, 1260.122.116, 1260.122.120, 1260.122.124, 1260.122.16, 1260.122.20, 1260.122.24, 1260.122.28, 1260.122.48, 1260.122.52, 1260.122.56, 1260.122.60, 1260.122.80, 1260.122.84, 1260.122.88, 1260.122.92


In [5]:
df_trips = pd.read_csv(data_path + 'trips.txt',
                       header=0,
                       encoding='utf-8-sig',
                       usecols=["route_id", "service_id", "trip_id", "trip_short_name"])
df_filtered_trips = df_trips[df_trips['service_id'].isin(todays_services)]
pd.options.display.max_rows = 10
print(df_trips)

       route_id    service_id                         trip_id  trip_short_name
0        BNK_2a    1260.122.4    1--A.1260.122.4.M.8.55188158              NaN
1        BNK_2a   1260.122.48   1--A.1260.122.48.M.8.55188157              NaN
2        BNK_2a   1260.122.64   1--A.1260.122.64.M.8.55188157              NaN
3        BNK_2a    1260.122.8    1--A.1260.122.8.M.8.55188158              NaN
4      RTTA_DEF  1603.103.128  1--A.1603.103.128.M.8.54724494              NaN
...         ...           ...                             ...              ...
57308   CTY_W2a    1620.100.2    WT28.1620.100.2.X.5.55042064              NaN
57309   CTY_W2a    1620.100.4    WT28.1620.100.4.X.5.55042062              NaN
57310   CTY_W2a   483.101.120   WT28.483.101.120.X.5.55277012              NaN
57311   CTY_W2a    487.111.60    WT28.487.111.60.X.5.55310994              NaN
57312   CTY_W2a    487.111.64    WT28.487.111.64.X.5.55310994              NaN

[57313 rows x 4 columns]


Gosh, that's a lot of services... What are RTTA_DEF and RTTA_REV... and are those CountryLink services??? Let's filter out some of these services as we're only going to analyse what is going on in the general Sydney commuter network.

In [6]:
ROUTES_TO_IGNORE = ["CTY_NC1", "CTY_NC1a", "CTY_NC2", 
                    "CTY_NW1a", "CTY_NW1b", "CTY_NW1c", "CTY_NW1d", "CTY_NW2a", "CTY_NW2b", 
                    "CTY_S1a", "CTY_S1b", "CTY_S1c", "CTY_S1d", "CTY_S1e", "CTY_S1f", 
                    "CTY_S1g", "CTY_S1h", "CTY_S1i", 
                    "CTY_S2a", "CTY_S2b", "CTY_S2c", "CTY_S2d", "CTY_S2e", "CTY_S2f", 
                    "CTY_S2g", "CTY_S2h", "CTY_S2i", 
                    "CTY_W1a", "CTY_W1b", "CTY_W2a", "CTY_W2b", 
                    "HUN_1a", "HUN_1b", "HUN_2a", "HUN_2b", 
                    "RTTA_DEF", "RTTA_REV"]
df_trips = df_trips[~df_trips['route_id'].isin(ROUTES_TO_IGNORE)]
print(df_trips)

      route_id   service_id                        trip_id  trip_short_name
0       BNK_2a   1260.122.4   1--A.1260.122.4.M.8.55188158              NaN
1       BNK_2a  1260.122.48  1--A.1260.122.48.M.8.55188157              NaN
2       BNK_2a  1260.122.64  1--A.1260.122.64.M.8.55188157              NaN
3       BNK_2a   1260.122.8   1--A.1260.122.8.M.8.55188158              NaN
6       BNK_2a  1603.103.60  1--A.1603.103.60.M.8.54724492              NaN
...        ...          ...                            ...              ...
57276    BMT_2   483.101.56   WN18.483.101.56.N.2.55278207              NaN
57277    BMT_2   483.101.64   WN18.483.101.64.N.2.55278207              NaN
57278    BMT_2    487.111.4    WN18.487.111.4.N.2.55308164              NaN
57279    BMT_2   487.111.56   WN18.487.111.56.N.2.55308164              NaN
57280    BMT_2   487.111.64   WN18.487.111.64.N.2.55308164              NaN

[47941 rows x 4 columns]


Now, the `stop_times.txt` file contains the stop files from across a number of days. We can filter out which services we want by only looking at trips that are running today.

In [7]:
df_stop_times = pd.read_csv(data_path + 'stop_times.txt', header=0,
                            encoding='utf-8-sig',
                            dtype={'stop_id': str},
                            usecols=["trip_id", "arrival_time", "departure_time", "stop_id"],
                            parse_dates=['arrival_time', 'departure_time'])

# remove any trips from stop_times that did NOT happen on this date
df_filtered_stop_times = df_stop_times[df_stop_times['trip_id'].isin(df_filtered_trips['trip_id'])]
print(df_filtered_stop_times)

                               trip_id arrival_time departure_time  stop_id
10       1--A.1260.122.48.M.8.55188157     03:52:00       03:52:00  2144243
11       1--A.1260.122.48.M.8.55188157     03:54:12       03:55:00  2141313
12       1--A.1260.122.48.M.8.55188157     03:57:30       03:57:30   214063
13       1--A.1260.122.48.M.8.55188157     03:58:42       03:58:42   214074
14       1--A.1260.122.48.M.8.55188157     04:01:24       04:01:24  2135234
...                                ...          ...            ...      ...
1033592  WT28.1260.122.60.X.5.55187037     20:26:24       20:26:24   214072
1033593  WT28.1260.122.60.X.5.55187037     20:27:36       20:29:12  2135232
1033594  WT28.1260.122.60.X.5.55187037     20:31:24       20:31:24   213491
1033595  WT28.1260.122.60.X.5.55187037     20:39:06       20:39:06  2015133
1033596  WT28.1260.122.60.X.5.55187037     20:42:24       23:34:00  2000325

[73135 rows x 4 columns]


## Parsing delay data
In the archive, all of the responses to requests to the [Transport for NSW Open Data API](https://opendata.transport.nsw.gov.au) have been saved. As part of the repository, there is a Python task for making these requests every two minutes, 24 hours a day.
This data is in the format according to [General Transit Feed Specification](https://developers.google.com/transit/) and can be parsed with the [GTFS python library](https://developers.google.com/transit/gtfs-realtime/examples/python-sample).

In [8]:
from google.transit import gtfs_realtime_pb2
import sys
sys.path.append('../')
from src.features.trip_objects import *
from src.features.trip_helper import *
import pickle

# get all the delay files
files_in_dir = sorted(glob.glob(data_path + '*.pickle'))

merged_delays = dict()

for delay_data_file in files_in_dir:
    try:
        current_delay_response = pickle.load(open(delay_data_file, "rb"))
    except Exception as e: 
        print(e)
        continue  # next file

    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(current_delay_response)
    for entity in feed.entity:
        if entity.HasField('trip_update') and len(entity.trip_update.stop_time_update) > 0:
            trip_update = TripUpdate(entity.trip_update.trip.trip_id,
                                     entity.trip_update.trip.route_id,
                                     entity.trip_update.trip.schedule_relationship,
                                     entity.trip_update.timestamp)

            for stop_time_update in entity.trip_update.stop_time_update:
                trip_update.stop_time_updates[stop_time_update.stop_id] \
                    = StopTimeUpdate(stop_time_update.stop_id,
                                     stop_time_update.arrival.delay,
                                     stop_time_update.departure.delay,
                                     stop_time_update.schedule_relationship)

            if trip_update is None:
                print('trip update is none')
            # merge with current trips
            if trip_update.trip_id in merged_delays:
                merged_delays[trip_update.trip_id] = \
                    merge_trips(merged_delays[trip_update.trip_id], trip_update)
            else:
                merged_delays[trip_update.trip_id] = trip_update

print("Found " + str(len(merged_delays)) + " trips")

Found 3515 trips


## Full schedule with delays
We want today's timetable, but with the delays on a per stop basis.
First, create the dataset, and add columns for delays and schedule relationship.

Merge the delays with the schedule, so they appear in the same dataset.

In [9]:
df_stop_times = df_filtered_stop_times

# insert actual arrival, actual departure, and cancellation states into data frame
# mark all as N/A to start with, so we know which things never had real time updates
df_stop_times.insert(2, 'arrival_delay', 'N/A')
df_stop_times.insert(3, 'actual_arrival_time', 'N/A')
df_stop_times.insert(5, 'departure_delay', 'N/A')
df_stop_times.insert(6, 'actual_departure_time', 'N/A')
df_stop_times.insert(7, 'schedule_relationship', 'N/A')

Now we go through each trip's delay information, then match up its stops with the corresponding row of the timetable.

In [10]:
# load all delays found on this date
trip_delays = merged_delays

df_trips = df_filtered_trips

# iterate through all the trips we 
for trip in trip_delays.values():
    if trip.trip_id not in df_trips['trip_id'].values:
        # print("Trip " + trip.trip_id + " was not supposed to run today!")
        continue

    for stop_time_update in trip.stop_time_updates.values():
        # some of these values might be 24:00, 25:00 etc to signify next day

        idx = df_stop_times[(df_stop_times['trip_id'] == trip.trip_id) &
                            (df_stop_times['stop_id'] == stop_time_update.stop_id)].index
        if idx.empty:
            # it shouldn't be
            continue

        idx = idx.item()

        # calculate the real time
        actual_arrival_time = update_time('20190123', df_stop_times.at[idx, 'arrival_time'],
                                               stop_time_update.arrival_delay)
        actual_departure_time = update_time('20190123', df_stop_times.at[idx, 'departure_time'],
                                                 stop_time_update.departure_delay)

        # add the new values to the new columns

        df_stop_times.at[idx, 'arrival_delay'] = stop_time_update.arrival_delay
        df_stop_times.at[idx, 'actual_arrival_time'] = actual_arrival_time
        df_stop_times.at[idx, 'departure_delay'] = stop_time_update.departure_delay
        df_stop_times.at[idx, 'actual_departure_time'] = actual_departure_time
        df_stop_times.at[idx, 'schedule_relationship'] = stop_time_update.schedule_relationship

df_filtered_stop_times_delays = df_stop_times
print(df_filtered_stop_times_delays)

                               trip_id arrival_time arrival_delay  \
10       1--A.1260.122.48.M.8.55188157     03:52:00           N/A   
11       1--A.1260.122.48.M.8.55188157     03:54:12           N/A   
12       1--A.1260.122.48.M.8.55188157     03:57:30           N/A   
13       1--A.1260.122.48.M.8.55188157     03:58:42           N/A   
14       1--A.1260.122.48.M.8.55188157     04:01:24           N/A   
...                                ...          ...           ...   
1033592  WT28.1260.122.60.X.5.55187037     20:26:24           N/A   
1033593  WT28.1260.122.60.X.5.55187037     20:27:36          7628   
1033594  WT28.1260.122.60.X.5.55187037     20:31:24           N/A   
1033595  WT28.1260.122.60.X.5.55187037     20:39:06           N/A   
1033596  WT28.1260.122.60.X.5.55187037     20:42:24          7836   

        actual_arrival_time departure_time departure_delay  \
10                      N/A       03:52:00             N/A   
11                      N/A       03:55:00     

## Summary of each service
Instead of having delay information for each stop of each service, add columns to the trip table that summarises the delays on that service.
First, add these columns.

In [11]:
# load the trip ids of that actual trips that happened on this day
df_trips = df_filtered_trips

# insert actual arrival, actual departure, and cancellation states into dataframe
# mark all as N/A to start with, so we know which things never had real time updates

df_trips.insert(0, 'start_timestamp', 'N/A')
df_trips.insert(1, 'end_timestamp', 'N/A')
df_trips.insert(5, 'maximum_arrival_delay', 0)
df_trips.insert(6, 'average_arrival_delay', 0)
df_trips.insert(7, 'maximum_departure_delay', 0)
df_trips.insert(8, 'average_departure_delay', 0)
df_trips.insert(9, 'schedule_relationship', 0)

Now match each trip's delay information with the corresponding row. To get the start and end timestamp of a service, grab that information from the stop time results as it was already calculated.

In [12]:
# get the stop times for trips that happened this day
df_stop_times = df_filtered_stop_times_delays

# load all delays found on this date
trip_delays = merged_delays

for trip in trip_delays.values():
    if trip.trip_id not in df_trips['trip_id'].values:
        # print("Trip " + trip.trip_id + " was not supposed to run today!")
        continue

    idx = df_trips[(df_trips['trip_id'] == trip.trip_id)].index
    if idx.empty:
        # it shouldn't be
        continue

    idx = idx.item()

    df_trips.at[idx, 'maximum_arrival_delay'] = trip.maximum_arrival_delay()
    df_trips.at[idx, 'average_arrival_delay'] = trip.average_arrival_delay()
    df_trips.at[idx, 'maximum_departure_delay'] = trip.maximum_departure_delay()
    df_trips.at[idx, 'average_departure_delay'] = trip.average_departure_delay()
    df_trips.at[idx, 'schedule_relationship'] = trip.overall_schedule_relationship()


for i in df_trips.index:
    departure_series = df_stop_times[df_stop_times['trip_id'] == df_trips.at[i, 'trip_id']]['departure_time']
    if len(departure_series) < 2:
        continue
    df_trips.at[i, 'start_timestamp'] = convert_to_timestamp('20190123', departure_series.iloc[0])
    df_trips.at[i, 'end_timestamp'] = convert_to_timestamp('20190123', departure_series.iloc[-1])

df_filtered_trips_delays = df_trips
print(df_filtered_trips_delays)

           start_timestamp        end_timestamp route_id   service_id  \
1      2019-01-23 03:52:00  2019-01-23 04:17:00   BNK_2a  1260.122.48   
19     2019-01-23 04:17:01  2019-01-23 04:58:00   APS_1a  1260.122.48   
37     2019-01-23 04:58:01  2019-01-23 05:17:00   APS_2a  1260.122.48   
55     2019-01-23 05:17:01  2019-01-23 05:58:00   APS_1a  1260.122.48   
73     2019-01-23 05:58:01  2019-01-23 06:35:00   APS_2a  1260.122.48   
...                    ...                  ...      ...          ...   
57232  2019-01-23 05:46:01  2019-01-23 11:02:00    BMT_2  1260.122.56   
57252  2019-01-23 17:47:01  2019-01-23 22:17:00    BMT_1  1260.122.56   
57267  2019-01-23 22:17:01  2019-01-23 23:26:00    BMT_2  1260.122.56   
57289  2019-01-23 07:19:01  2019-01-23 14:15:00  CTY_W1a  1260.122.60   
57301  2019-01-23 14:15:01  2019-01-23 23:34:00  CTY_W2a  1260.122.60   

                             trip_id  maximum_arrival_delay  \
1      1--A.1260.122.48.M.8.55188157                      0 

What were the worst delays?

In [13]:
df_filtered_trips_delays.sort_values(by=['maximum_departure_delay'], ascending=False).head(n=10)

,start_timestamp,end_timestamp,route_id,service_id,trip_id,maximum_arrival_delay,average_arrival_delay,maximum_departure_delay,average_departure_delay,schedule_relationship,trip_short_name
57301,2019-01-23 14:15:01,2019-01-23 23:34:00,CTY_W2a,1260.122.60,WT28.1260.122.60.X.5.55187037,7836,2550,7562,2106,0,NaN
56746,2019-01-23 18:33:01,2019-01-23 21:30:00,BMT_1,1260.122.48,W581.1260.122.48.V.4.55187453,5056,2214,5056,2358,0,NaN
47062,2019-01-23 07:32:01,2019-01-23 07:44:00,RTTA_REV,1260.122.112,HT23.1260.122.112.X.7.55186340,4530,4530,4530,4530,0,NaN
56457,2019-01-23 15:01:00,2019-01-23 18:02:00,BMT_2,1260.122.48,W560.1260.122.48.V.4.55186785,3894,2985,3894,2964,0,NaN
21249,2019-01-23 06:28:30,2019-01-23 15:02:00,CTY_W2b,1260.122.16,3AS8.1260.122.16.R.29.55187900,4015,2007,2545,1272,0,NaN
56852,2019-01-23 21:18:01,2019-01-23 23:40:48,BMT_1,1260.122.48,W591.1260.122.48.V.4.55187824,3951,559,2508,323,0,NaN
52749,2019-01-23 14:15:01,2019-01-23 17:24:00,CCN_1a,1260.122.60,N155.1260.122.60.V.8.55187399,2512,2159,2482,2103,0,NaN
23116,2019-01-23 15:02:01,2019-01-23 19:02:36,CTY_W1b,1260.122.16,4SA8.1260.122.16.R.29.55187901,7025,3898,2260,1506,0,NaN
56689,2019-01-23 17:31:19,2019-01-23 19:54:30,BMT_1,1260.122.124,W575.1260.122.124.V.8.55191035,1706,623,2174,702,0,NaN
45859,2019-01-23 22:11:01,2019-01-23 22:29:24,RTTA_REV,1260.122.60,H181.1260.122.60.V.8.55186522,2700,1427,2106,1105,0,NaN
